# 1. Using OpenAlex to find taxonomists

## 1.2. Retrieving articles from taxonomic journals

The OpenAlex API was used to retrieve all recent articles (2013-2022) published in taxonomic journals. A list of journals was obtained via a SPARQL query on Wikidata, with subjects such as "taxonomy", "phylogeny", "systematics" and "biological nomenclature" and another list of journals via OpenAlex by searching for sources with the concept "taxonomy" associated with it. The articles from these journals were retrieved with the OpenAlex API. This data contains information on the authors and associated institutions, publishers, and abstract, among other things.

Since we want to find taxonomists for all animals, fungi and plants, searching for each order becomes unwieldy and potentially contentious since determining which orders should be searched could be difficult. Even in the insect example, Isoptera for example is often seen as an infraorder. Moreover, different clades may be studied (and named in articles) at different taxonomic levels, for example the family level is more important for botanists than the order. Even within insect taxonomy, Coleoptera and many other orders are far too large and expertise must be studied at lower clades.

Instead, we start from a list of taxonomic journals an retrieve their articles, then filter these articles for relevant content.

A list of journals was found by searching OpenAlex for sources with taxonomy as an associated concept, and by searching Wikidata for journals (instance of = scientific journal OR academic journal) whose main subject or field of study is taxonomy, biological nomenclature, phylogenetics, or related concepts.  

In [7]:
import numpy as np
import pandas as pd
import pickle
import time
import openalex
import requests

In [3]:
# search wikidata for journals
query = openalex.build_sparql_query(["Q8269924", # taxonomy
                                     "Q11398", # biological classification
                                     "Q1138178", # plant taxonomy
                                     "Q1469725", # animal taxonomy
                                     "Q3516404", # systematics
                                     "Q522190", # biological nomenclature
                                     "Q3310776", # botanical nomenclature
                                     "Q3343211"]) # zoological nomenclature
query2 = openalex.build_sparql_query(["Q171184", # phylogenetics
                                      "Q115135896"]) # animal phylogeny

results_df = openalex.get_sparql_results(query)
results_df2 = openalex.get_sparql_results(query2)

wd_journals = pd.concat([results_df, results_df2], ignore_index=True)
wd_journals

,item,issn_l,issn,itemLabel
0,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '1313-2970'}","{'type': 'literal', 'value': '1313-2970'}","{'type': 'literal', 'value': 'Q219980'}"
1,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '1313-2970'}","{'type': 'literal', 'value': '1313-2989'}","{'type': 'literal', 'value': 'Q219980'}"
2,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '1759-6831'}","{'type': 'literal', 'value': '1674-4918'}","{'type': 'literal', 'value': 'Q15733644'}"
3,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '1759-6831'}","{'type': 'literal', 'value': '1759-6831'}","{'type': 'literal', 'value': 'Q15733644'}"
4,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '2118-9773'}","{'type': 'literal', 'value': '2118-9773'}","{'type': 'literal', 'value': 'Q21386186'}"
5,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '1175-5326'}","{'type': 'literal', 'value': '1175-5326'}","{'type': 'literal', 'value': 'Q220370'}"
6,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '1175-5326'}","{'type': 'literal', 'value': '1175-5334'}","{'type': 'literal', 'value': 'Q220370'}"
7,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '0367-5041'}","{'type': 'literal', 'value': '0367-5041'}","{'type': 'literal', 'value': 'Q94579799'}"
8,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '0367-5041'}","{'type': 'literal', 'value': '1989-953X'}","{'type': 'literal', 'value': 'Q94579799'}"
9,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'literal', 'value': '2399-3421'}","{'type': 'literal', 'value': '2399-3421'}","{'type': 'literal', 'value': 'Q58315628'}"


In [4]:
# get ISSN-L of each journal
# a journal may have several ISSNs but only one ISSN-L, which is always the same as one of its ISSNs
issn_wikidata = [] 
for journal in wd_journals["issn_l"]:
    issn_wikidata.append(journal["value"])
    
issn_wikidata

['1313-2970',
 '1313-2970',
 '1759-6831',
 '1759-6831',
 '2118-9773',
 '1175-5326',
 '1175-5326',
 '0367-5041',
 '0367-5041',
 '2399-3421',
 '1063-5157',
 '1063-5157',
 '0511-9618',
 '0511-9618',
 '0075-5974',
 '0075-5974',
 '1435-1935',
 '1435-1935',
 '1346-7565',
 '1346-7565',
 '1225-8318',
 '1225-8318',
 '1394-5130',
 '0132-8069',
 '0307-6970',
 '0307-6970',
 '1179-7649',
 '1179-7649',
 '1030-1887',
 '1030-1887',
 '0085-4417',
 '0085-4417',
 '1055-7903',
 '1055-7903',
 '2373-7697',
 '1179-3155',
 '1179-3155',
 '1314-2003',
 '1314-2003',
 '2234-6953',
 '2234-6953',
 '0300-3256',
 '0300-3256',
 '1314-2488',
 '1314-2488',
 '0007-5167',
 '0007-5167',
 '0376-4443',
 '2703-3082',
 '2703-3082',
 '0004-8038',
 '0004-8038',
 '0077-1813',
 '0077-1813',
 '1522-8339',
 '1522-8339',
 '2095-6827',
 '1055-7903',
 '1055-7903']

In [5]:
# the journal Taxon could not be found this way, but it is an important player in taxonomy
issn_wikidata.append("0040-0262") # Taxon

In [8]:
# search OpenAlex for journals
tax_journals = requests.get("https://api.openalex.org/sources?filter=concepts.id:C58642233&per_page=200")
tax_journals = pd.DataFrame(tax_journals.json()["results"])
tax_journals

,id,issn_l,issn,display_name,host_organization,host_organization_name,host_organization_lineage,works_count,cited_by_count,summary_stats,...,country_code,societies,alternate_titles,abbreviated_title,type,x_concepts,counts_by_year,works_api_url,updated_date,created_date
0,https://openalex.org/S171471881,1175-5326,"[1175-5334, 1175-5326]",Zootaxa,https://openalex.org/P4310321855,Q15088586,[https://openalex.org/P4310321855],41493,179047,"{'2yr_mean_citedness': 0.493234100135318, 'h_i...",...,NZ,[],[],None,journal,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 1046, 'cited_by...",https://api.openalex.org/works?filter=primary_...,2023-06-12,2016-06-24
1,https://openalex.org/S100787194,1179-3155,"[1179-3163, 1179-3155]",Phytotaxa,https://openalex.org/P4310321855,Q15088586,[https://openalex.org/P4310321855],11235,34673,"{'2yr_mean_citedness': 0.5481639169771155, 'h_...",...,NZ,[],[],None,journal,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 422, 'cited_by_...",https://api.openalex.org/works?filter=primary_...,2023-06-12,2016-06-24
2,https://openalex.org/S199213172,1313-2970,"[1313-2970, 1313-2989]",ZooKeys,https://openalex.org/P4310321056,Pensoft Publishers,[https://openalex.org/P4310321056],6729,42766,"{'2yr_mean_citedness': 1.1759603469640645, 'h_...",...,BG,[],[],None,journal,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 186, 'cited_by_...",https://api.openalex.org/works?filter=primary_...,2023-06-12,2016-06-24
3,https://openalex.org/S44318362,0093-4666,"[0093-4666, 2154-8889]",Mycotaxon,https://openalex.org/P4310322165,Mycotaxon Publications,[https://openalex.org/P4310322165],4420,16468,"{'2yr_mean_citedness': 0.5909090909090909, 'h_...",...,US,[],[],None,journal,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=primary_...,2023-06-12,2016-06-24
4,https://openalex.org/S2764437286,0007-5167,"[2057-0570, 0007-5167]",The Bulletin of zoological nomenclature,https://openalex.org/P4310322357,International Trust for Zoological Nomenclature,[https://openalex.org/P4310322357],3793,2682,"{'2yr_mean_citedness': 0.12149532710280374, 'h...",...,GB,[],[],None,journal,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2023, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=primary_...,2023-06-12,2017-10-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,https://openalex.org/S4306489654,None,None,Śląski Uniwersytet Medyczny w Katowicach/Medic...,None,None,[],2,0,"{'2yr_mean_citedness': 0.0, 'h_index': 0, 'i10...",...,None,[],[],None,ebook platform,"[{'id': 'https://openalex.org/C18903297', 'wik...","[{'year': 2021, 'works_count': 2, 'cited_by_co...",https://api.openalex.org/works?filter=primary_...,2023-06-07,2022-10-17
158,https://openalex.org/S4306490621,None,None,VNIIP – FSC VIEV eBooks,None,None,[],2,0,"{'2yr_mean_citedness': 0.0, 'h_index': 0, 'i10...",...,None,[],[],None,ebook platform,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2022, 'works_count': 2, 'cited_by_co...",https://api.openalex.org/works?filter=primary_...,2023-06-07,2022-10-17
159,https://openalex.org/S4210209646,0096-3852,[0096-3852],Transactions of the Academy of Science of Sain...,None,None,[],1,1,"{'2yr_mean_citedness': 0.0, 'h_index': 1, 'i10...",...,US,[],"[Trans. Acad. Sci. of St. Louis, Trans. St. Lo...",None,journal,"[{'id': 'https://openalex.org/C18903297', 'wik...","[{'year': 2014, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=primary_...,2023-06-08,2022-02-03
160,https://openalex.org/S4306485908,None,None,All-Russian Scientific Research Institute for ...,None,None,[],1,0,"{'2yr_mean_citedness': 0.0, 'h_index': 0, 'i10...",...,None,[],[],None,ebook platform,"[{'id': 'https://openalex.org/C18903297', 'wik...","[{'year': 2021, 'works_count': 1,

In [9]:
# WikiData and OpenAlex journals all in one neat list by ISSN-L
issns = list(filter(None, tax_journals["issn_l"])) # remove None values
issns.extend(issn_wikidata)
issns = set(issns) # remove duplicates
len(issns)

146

In [13]:
# get all articles from taxonomic journals
articles = []

for issn in issns:
    start = time.time()
    
    # OpenAlex can't search ISSN-L, but ISSN works just as well
    journal_articles = openalex.request_works("locations.source.issn:"+issn, to_date="2022-12-31")
    articles.append(journal_articles)
    
    end = time.time()
    print(issn+" done in "+str(end-start)+" seconds")

articles = pd.concat(articles)
articles

Number of publications for locations.source.issn:0024-0672: 8
0024-0672 done in 0.6248579025268555 seconds
Number of publications for locations.source.issn:1445-5226: 409
1445-5226 done in 2.525312662124634 seconds
Number of publications for locations.source.issn:2719-1486: 13
2719-1486 done in 0.5716807842254639 seconds
Number of publications for locations.source.issn:0394-6975: 118
0394-6975 done in 1.101125717163086 seconds
Number of publications for locations.source.issn:0968-0446: 0
0968-0446 done in 0.3668708801269531 seconds
Number of publications for locations.source.issn:0075-5974: 660
0075-5974 done in 2.6435720920562744 seconds
Number of publications for locations.source.issn:1560-2745: 367
1560-2745 done in 2.5468714237213135 seconds
Number of publications for locations.source.issn:0341-0145: 29
0341-0145 done in 0.669776439666748 seconds
Number of publications for locations.source.issn:0065-8170: 1
0065-8170 done in 0.519942045211792 seconds
Number of publications for loca

Number of publications for locations.source.issn:2373-7697: 0
2373-7697 done in 0.36979031562805176 seconds
Number of publications for locations.source.issn:1139-5192: 13
1139-5192 done in 0.6603565216064453 seconds
Number of publications for locations.source.issn:2363-7684: 0
2363-7684 done in 0.37003040313720703 seconds
Number of publications for locations.source.issn:0002-1474: 0
0002-1474 done in 0.3797593116760254 seconds
Number of publications for locations.source.issn:0749-6737: 413
0749-6737 done in 3.1002299785614014 seconds
Number of publications for locations.source.issn:0024-1652: 0
0024-1652 done in 0.3584136962890625 seconds
Number of publications for locations.source.issn:1477-2000: 593
1477-2000 done in 2.8649063110351562 seconds
Number of publications for locations.source.issn:0399-0974: 40
0399-0974 done in 0.848111629486084 seconds
Number of publications for locations.source.issn:1340-3540: 640
1340-3540 done in 2.8367204666137695 seconds
Number of publications for l

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,grants,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,is_authors_truncated
0,https://openalex.org/W2097461610,None,A systematic monograph of the Recent Pentastom...,A systematic monograph of the Recent Pentastom...,2013,2013-01-01,{'openalex': 'https://openalex.org/W2097461610...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],[],"[https://openalex.org/W1451478857, https://ope...",https://api.openalex.org/works/W2097461610/ngrams,"{'We': [0], 'compile': [1], 'all': [2], 'publi...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2020, 'cited_by_count': 1}, {'year':...",2023-06-01T01:06:38.350874,2016-06-24,NaN
1,https://openalex.org/W2977549230,None,Microcionid sponges from Northwest Africa and ...,Microcionid sponges from Northwest Africa and ...,2013,2013-01-01,{'openalex': 'https://openalex.org/W2977549230...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],[],"[https://openalex.org/W363939732, https://open...",https://api.openalex.org/works/W2977549230/ngrams,None,https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2014, 'cited_by_count': 7}]",2023-05-27T00:23:19.133568,2019-10-10,NaN
2,https://openalex.org/W2098184878,None,Mycale species (Porifera: Poecilosclerida) of ...,Mycale species (Porifera: Poecilosclerida) of ...,2014,2014-01-01,{'openalex': 'https://openalex.org/W2098184878...,en,"{'is_oa': False, 'landing_page_url': 'http://w...",journal-article,...,[],"[https://openalex.org/W5267670, https://openal...","[https://openalex.org/W1870150722, https://ope...",https://api.openalex.org/works/W2098184878/ngrams,"{'Based': [0], 'on': [1], 'collections': [2, 5...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2017, 'cited_by_count': 1}, {'year':...",2023-05-26T23:14:12.444095,2016-06-24,NaN
3,https://openalex.org/W1841372523,None,Dinotrema brevis spec. nov. (Hymenoptera: Brac...,Dinotrema brevis spec. nov. (Hymenoptera: Brac...,2014,2014-01-01,{'openalex': 'https://openalex.org/W1841372523...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W38046216, https://opena...","[https://openalex.org/W893905258, https://open...",https://api.openalex.org/works/W1841372523/ngrams,"{'A': [0, 17], 'new': [1], 'species': [2, 21],...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2019, 'cited_by_count': 1}]",2023-05-27T02:13:06.320098,2016-06-24,NaN
4,https://openalex.org/W813610620,None,Sigalphus anjae spec. nov. (Hymenoptera: Braco...,Sigalphus anjae spec. nov. (Hymenoptera: Braco...,2014,2014-01-01,{'openalex': 'https://openalex.org/W813610620'...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W1537125554, https://ope...",[],https://api.openalex.org/works/W813610620/ngrams,"{'A': [0, 20], 'new': [1], 'species': [2, 25],...",https://api.openalex.org/works?filter=cites:W8...,"[{'year': 2016, 'cited_by_count': 1}]",2023-05-26T07:36:47.829660,2016-06-24,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,https://openalex.org/W4316923294,https://doi.org/10.21684/0132-8077-2022-30-2-1...,THE MITE GENUS HISTIOGASTER (ACARI: ACARIDAE) ...,THE MITE GENUS HISTIOGASTER (ACARI: ACARIDAE) ...,2022,2022-01-01,{'openalex': 'https://openalex.org/W4316923294...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",journal-article,...,[],"[https://openalex.org/W1550069160, https://ope...","[https://openalex.org/W1977168516, https://ope...",https://api.openalex.org/works/W4316923294/ngrams,"{'Two': [0], 'new': [1], 'species': [2, 61, 75...",https://api.openalex.org/works?filter=cites:W4...,[],2023-05-30T07:43:48.170462,2023-01-18,NaN
149,https://openalex.org/W4316923321,https://doi.org/10.21684/0132-8077-2022-30-2-2...,A N

In [14]:
articles.to_pickle("./data/articles_global_with_taxonomy_concept_in_journal.pkl")

In [16]:
# filter for EU articles
eu_articles = openalex.filter_eu_articles(articles)
eu_articles

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,grants,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,is_authors_truncated
0,https://openalex.org/W2128055154,https://doi.org/10.1071/is13061,"New insights into the phylogeny, systematics a...","New insights into the phylogeny, systematics a...",2014,2014-01-01,{'openalex': 'https://openalex.org/W2128055154...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W102715436, https://open...","[https://openalex.org/W1554287491, https://ope...",https://api.openalex.org/works/W2128055154/ngrams,"{'Although': [0], 'some': [1, 22], 'clades': [...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 4}, {'year':...",2023-06-08T11:46:51.791335,2016-06-24,NaN
1,https://openalex.org/W2282635459,https://doi.org/10.1071/is15033,Integrative systematic studies on tardigrades ...,Integrative systematic studies on tardigrades ...,2016,2016-08-31,{'openalex': 'https://openalex.org/W2282635459...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W133481079, https://open...","[https://openalex.org/W1816690434, https://ope...",https://api.openalex.org/works/W2282635459/ngrams,"{'Tardigrades': [0], 'represent': [1], 'one': ...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 1}, {'year':...",2023-05-29T00:22:56.109044,2016-06-24,NaN
2,https://openalex.org/W2415892372,https://doi.org/10.1071/is15045,Genetic diversity and biogeography of the sout...,Genetic diversity and biogeography of the sout...,2016,2016-01-01,{'openalex': 'https://openalex.org/W2415892372...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W177145487, https://open...","[https://openalex.org/W1985231932, https://ope...",https://api.openalex.org/works/W2415892372/ngrams,"{'Antarctica': [0, 130], 'is': [1, 24], 'an': ...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 3}, {'year':...",2023-06-09T23:39:27.926736,2016-06-24,NaN
3,https://openalex.org/W1534942643,https://doi.org/10.1071/is13002,Morphological and molecular insights on Megalo...,Morphological and molecular insights on Megalo...,2013,2013-01-01,{'openalex': 'https://openalex.org/W1534942643...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W19893924, https://opena...","[https://openalex.org/W1534942643, https://ope...",https://api.openalex.org/works/W1534942643/ngrams,"{'The': [0, 73, 92], 'genus': [1], 'Megalothor...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2023, 'cited_by_count': 4}, {'year':...",2023-06-10T05:55:09.259272,2016-06-24,NaN
4,https://openalex.org/W2159262135,https://doi.org/10.1071/is13030,Two distinct evolutionary lineages of the Asta...,Two distinct evolutionary lineages of the Asta...,2014,2014-01-01,{'openalex': 'https://openalex.org/W2159262135...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[],"[https://openalex.org/W247704383, https://open...","[https://openalex.org/W1985896783, https://ope...",https://api.openalex.org/works/W2159262135/ngrams,"{'Narrow-clawed': [0], 'crayfish': [1, 13, 85]...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2023, 'cited_by_count': 2}, {'year':...",2023-06-07T02:46:48.359217,2016-06-24,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17947,https://openalex.org/W4253285772,https://doi.org/10.21684/0132-8077-2019-27-2-1...,"MERISTACARUS BOCHKOVI (ACARI, ORIBATIDA, LOHMA...","MERISTACARUS BOCHKOVI (ACARI, ORIBATIDA, LOHMA...",2019,2019-12-30,{'openalex': 'https://openalex.org/W4253285772...,en,"{'is_oa': False, 'landing_page_url': 'https://...",journal-article,...,[{'funder': 'http

In [17]:
eu_articles.to_pickle("./data/articles_EU27_with_taxonomy_concept_in_journal.pkl")